In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
import math
import random
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score,roc_auc_score
import warnings
warnings.filterwarnings("ignore")

In [14]:
data1 = pd.read_csv("../../dataset/NASA/PC4.csv")
data1.head()

,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,...,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,c
0,17,11,5,2,8,20,6,0.25,10,2,...,25,0.11,53,49,23,12,57,31.25,24,False
1,2,9,3,0,1,16,5,0.56,6,2,...,14,0.36,13,24,7,14,14,10.00,9,False
2,2,5,1,1,1,6,3,0.17,2,3,...,7,0.13,16,28,9,14,23,10.53,18,False
3,4,5,1,0,0,8,3,0.30,4,2,...,10,0.19,13,16,10,9,16,0.00,10,False
4,7,5,1,3,0,0,3,0.15,0,0,...,10,0.11,26,46,7,7,28,15.00,20,False


In [15]:
data1.describe()

,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,...,MULTIPLE_CONDITION_COUNT,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL
count,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,...,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000,1458.000000
mean,6.997257,8.449246,2.320302,2.109053,4.863512,7.235940,4.839506,0.329307,3.299040,0.967764,...,3.661866,15.909465,0.197997,38.707133,62.783265,13.110425,11.769547,38.047325,17.477455,20.613855
std,9.847289,12.715911,2.891473,5.317359,9.487406,16.192918,6.668257,0.329457,7.508203,1.074917,...,8.359637,21.269652,0.223384,71.446722,103.999833,20.535942,6.616703,49.828159,21.209440,25.234081
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.160000,0.000000,0.000000,...,0.000000,6.000000,0.080000,9.000000,15.000000,5.000000,7.000000,12.000000,0.000000,6.000000
50%,3.000000,5.000000,1.000000,0.000000,0.000000,0.000000,3.000000,0.250000,0.000000,0.000000,...,0.000000,10.000000,0.140000,19.000000,33.000000,9.000000,11.000000,21.000000,8.700000,12.000000
75%,10.000000,10.000000,3.000000,2.000000,6.000000,8.000000,6.000000,0.410000,4.000000,2.000000,...,4.000000,19.000000,0.230000,44.000000,73.000000,16.000000,15.000000,48.000000,30.770000,26.000000
max,90.000000,187.000000,24.000000,68.000000,78.000000,372.000000,94.000000,8.550000,186.000000,5.000000,...,186.000000,286.000000,2.290000,1403.000000,1687.000000,601.000000,38.000000,802.000000,93.020000,210.000000


In [16]:
data1.values.shape

(1458, 38)

In [17]:
data2=data1.values
for i in range(1458):
    if data2[i][37]==True:
        data2[i][37]=1
    else:
        data2[i][37]=0
data2[8]

array([36, 21, 1, 18, 43, 38, 11, 0.16, 18, 2, 3, 0.27, 42, 1, 0, 51, 0,
       15.89, 87.82, 122506.36, 0.47, 279, 0.01, 6805.91, 1395.0, 0.09,
       10, 19, 33, 0.07, 92, 187, 11, 21, 150, 54.46, 69, 1], dtype=object)

In [18]:
X=np.array(data2).astype(float)  

In [19]:
X[:,:37]

array([[17.  , 11.  ,  5.  , ..., 57.  , 31.25, 24.  ],
       [ 2.  ,  9.  ,  3.  , ..., 14.  , 10.  ,  9.  ],
       [ 2.  ,  5.  ,  1.  , ..., 23.  , 10.53, 18.  ],
       ...,
       [ 0.  ,  1.  ,  0.  , ...,  1.  ,  0.  ,  0.  ],
       [ 8.  ,  9.  ,  0.  , ..., 32.  , 45.45, 12.  ],
       [ 4.  ,  1.  ,  1.  , ..., 11.  , 20.  ,  4.  ]])

In [20]:
y=X[:,37]

In [21]:
X=X[:,0:37]

In [22]:
from imblearn.over_sampling import RandomOverSampler,SMOTE
#os =  RandomOverSampler()
X_res, y_res = SMOTE().fit_resample(X, y)
X_res.shape,y_res.shape

((2560, 37), (2560,))

In [23]:
X_res.shape
#X_res=X_res.reshape(898,21,1)
#X_res.shape

(2560, 37)

In [24]:
import numpy as np
from sklearn import preprocessing
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import datetime

def show_accuracy(predictLabel,Label):
    Label = np.ravel(Label).tolist()
    predictLabel = predictLabel.tolist()
    count = 0
    for i in range(len(Label)):
        if Label[i] == predictLabel[i]:
            count += 1
    return (round(count/len(Label),5))

class node_generator(object):
    def __init__(self, whiten = False):
        self.Wlist = []
        self.blist = []
        self.function_num = 0
        self.whiten = whiten

    def sigmoid(self, x):
        return 1.0/(1 + np.exp(-x))

    def relu(self, x):
        return np.maximum(x, 0)

    def tanh(self, x):
        return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

    def linear(self, x):
        return x

    def orth(self, W):
        """
        目前看来，这个函数应该配合下一个generator函数是生成权重的
        """
        for i in range(0, W.shape[1]):
            w = np.mat(W[:,i].copy()).T
            w_sum = 0
            for j in range(i):
                wj = np.mat(W[:,j].copy()).T
                w_sum += (w.T.dot(wj))[0,0]*wj
            w -= w_sum
            w = w/np.sqrt(w.T.dot(w))
            W[:,i] = np.ravel(w)

        return W

    def generator(self, shape, times):
        for i in range(times):
            random.seed(i)
            W = 2*np.random.random(size=shape)-1
            if self.whiten == True:
                W = self.orth(W)   # 只在增强层使用
            b = 2*np.random.random() -1
            yield (W, b)

    def generator_nodes(self, data, times, batchsize, function_num):
        # 按照bls的理论，mapping layer是输入乘以不同的权重加上不同的偏差之后得到的
        # 若干组，所以，权重是一个列表，每一个元素可作为权重与输入相乘
        self.Wlist = [elem[0] for elem in self.generator((data.shape[1], batchsize), times)]
        self.blist = [elem[1] for elem in self.generator((data.shape[1], batchsize), times)]

        self.function_num = {'linear':self.linear,
                        'sigmoid': self.sigmoid,
                        'tanh':self.tanh,
                        'relu':self.relu }[function_num]  # 激活函数供不同的层选择
        # 下面就是先得到一组mapping nodes，再不断叠加，得到len(Wlist)组mapping nodes
        nodes = self.function_num(data.dot(self.Wlist[0]) + self.blist[0])
        for i in range(1, len(self.Wlist)):
            nodes = np.column_stack((nodes, self.function_num(data.dot(self.Wlist[i])+self.blist[i])))
        return nodes

    def transform(self,testdata):
        testnodes = self.function_num(testdata.dot(self.Wlist[0])+self.blist[0])
        for i in range(1,len(self.Wlist)):
            testnodes = np.column_stack((testnodes, self.function_num(testdata.dot(self.Wlist[i])+self.blist[i])))
        return testnodes

class scaler:
    def __init__(self):
        self._mean = 0
        self._std = 0
    
    def fit_transform(self,traindata):
        self._mean = traindata.mean(axis = 0)
        self._std = traindata.std(axis = 0)
        return (traindata-self._mean)/(self._std+0.001)
    
    def transform(self,testdata):
        return (testdata-self._mean)/(self._std+0.001)

class broadnet:
    def __init__(self, 
                 maptimes = 10, 
                 enhencetimes = 10,
                 map_function = 'linear',
                 enhence_function = 'linear',
                 batchsize = 'auto', 
                 reg = 0.001):
        
        self._maptimes = maptimes
        self._enhencetimes = enhencetimes
        self._batchsize = batchsize
        self._reg = reg
        self._map_function = map_function
        self._enhence_function = enhence_function
        
        self.W = 0
        self.pesuedoinverse = 0
        self.normalscaler = scaler()
        self.onehotencoder = preprocessing.OneHotEncoder(sparse = False)
        self.mapping_generator = node_generator()
        self.enhence_generator = node_generator(whiten = True)

    def fit(self,data,label,weight):
        if self._batchsize == 'auto':
            self._batchsize = data.shape[1]
        data = self.normalscaler.fit_transform(data)
        label = self.onehotencoder.fit_transform(np.mat(label).T)
        
        mappingdata = self.mapping_generator.generator_nodes(data,self._maptimes,self._batchsize,self._map_function)
        enhencedata = self.enhence_generator.generator_nodes(mappingdata,self._enhencetimes,self._batchsize,self._enhence_function)
        
        #print('number of mapping nodes {0}, number of enhence nodes {1}'.format(mappingdata.shape[1],enhencedata.shape[1]))
        #print('mapping nodes maxvalue {0} minvalue {1} '.format(round(np.max(mappingdata),5),round(np.min(mappingdata),5)))
        #print('enhence nodes maxvalue {0} minvalue {1} '.format(round(np.max(enhencedata),5),round(np.min(enhencedata),5)))
        
        inputdata = np.column_stack((mappingdata,enhencedata))
        pesuedoinverse = self.pinv(inputdata,self._reg,weight)
        self.W =  pesuedoinverse.dot(label)
        
        #print('W:', self.W)
        #print('W:', self.W.shape)  
    
    #改写伪逆矩阵算法，将权重输入
    def pinv(self,A,reg,weight):
        return np.mat(reg*np.eye(A.shape[1])+A.T.dot(weight).dot(A)).I.dot(A.T).dot(weight)
    
    def decode(self,Y_onehot):
        Y = []
        for i in range(Y_onehot.shape[0]):
            lis = np.ravel(Y_onehot[i,:]).tolist()
            Y.append(lis.index(max(lis)))
        return np.array(Y)
    
    def accuracy(self,predictlabel,label):
        label = np.ravel(label).tolist()
        predictlabel = predictlabel.tolist()
        count = 0
        for i in range(len(label)):
            if label[i] == predictlabel[i]:
                count += 1
        return (round(count/len(label),5))
        
    def predict(self,testdata):
        testdata = self.normalscaler.transform(testdata)
        test_mappingdata = self.mapping_generator.transform(testdata)
        test_enhencedata = self.enhence_generator.transform(test_mappingdata)
        
        test_inputdata = np.column_stack((test_mappingdata,test_enhencedata)) 
        #print('*predictlabel shape:',self.decode(test_inputdata.dot(self.W)).shape)
        #print('*predictlabel:', self.decode(test_inputdata.dot(self.W)))
        #print('*accuracy:',show_accuracy(self.decode(test_inputdata.dot(self.W)),testlabel))
        return self.decode(test_inputdata.dot(self.W))
    
    def addWeight(self,trainlabel):
        #对BLS设置加权，按照训练集中的label比例进行权重设置
        #求训练数据集中label为1的个数
        count=0
        for z in range(len(trainlabel)):
            #print ('个数 %d ' %z)
            #print('k_train_label:', k_train_label[z])
            if trainlabel[z]==1:
                count=count+1
        #print ('label为1的个数 %d ' %j)
        #print(count)
        #print(k_train_label.shape)
        #print(len(k_train_label))
        #权重设置 
        weight = np.zeros((len(trainlabel),len(trainlabel)))
        for i in range(len(trainlabel)):
            if trainlabel[i]==1:
                #weight[i,i]=0.618/count
                weight[i,i]=2*(len(trainlabel)-count)/len(trainlabel)
            else:
                #weight[i,i]=1/(len(k_train_label)-count)
                weight[i,i]=2*count/len(trainlabel)
        #print('Weight:', weight)
        return weight

In [25]:
#将数据集划分为训练集和测试集
traindata,testdata,trainlabel,testlabel = train_test_split(X_res,y_res,test_size=0.25,random_state = 0)
#print(traindata.shape,trainlabel.shape,testdata.shape,testlabel.shape)
#X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2)
#y_train = np_utils.to_categorical(y_train)
#y_test = np_utils.to_categorical(y_test)

In [26]:
k_acc_list, k_f1_list, k_auc_list, k_recall_list, k_mcc_list, k_Gm_list=[],[],[],[],[],[]
#这里设置shuffle设置为ture就是打乱顺序在分配
kf = KFold(n_splits=3,shuffle=True,random_state=42)
for map_times in np.arange(15,36,5):
    acc_list_tmp, f1_list_tmp, auc_list_tmp, recall_list_tmp, mcc_list_tmp,Gm_list_tmp=[],[],[],[],[],[]
    for enhance_times in np.arange(15, 36,5):
        for k, (train, test) in enumerate(kf.split(traindata, trainlabel)):
            # kf.split输出的是索引，所以由索引获取交叉后的训练集和测试集及标签
            k_train_data,k_train_label = traindata[train], trainlabel[train]
            k_test_data,k_test_label = traindata[test], trainlabel[test]           

            bls = broadnet(maptimes = map_times, 
                       enhencetimes = enhance_times,
                       map_function = 'relu',
                       enhence_function = 'relu',
                       batchsize =100,
                       reg = 0.001)

            #根据训练数据设置权重
            weight=bls.addWeight(k_train_label)

            #训练
            starttime = datetime.datetime.now()
            bls.fit(k_train_data,k_train_label,weight)
            endtime = datetime.datetime.now()
            #print('the training time of BLS is {0} seconds'.format((endtime - starttime).total_seconds()))

            #print('k_test_label:', k_test_label)
            #预测
            k_predict_label = bls.predict(k_test_data)
            #print('k_predict_label:', k_predict_label)

            #评价指标计算
            acc=accuracy_score(k_test_label,k_predict_label,normalize=True)
            fmeasure=f1_score(k_test_label,k_predict_label, average='weighted', labels=np.unique(k_test_label))
            try:
                auc=roc_auc_score(k_test_label,k_predict_label, average='macro', sample_weight=None)
            except ValueError:
                pass
            recall=recall_score(k_test_label, k_predict_label, average='weighted')
            MCC=matthews_corrcoef(k_test_label,k_predict_label)
            Gmeasure=geometric_mean_score(k_test_label,k_predict_label, average='weighted')

            #将此次的十折交叉验证的结果 (10个)保存到pi_list_tmp中
            acc_list_tmp.append(acc)
            f1_list_tmp.append(fmeasure)
            auc_list_tmp.append(auc)
            recall_list_tmp.append(recall)
            mcc_list_tmp.append(MCC)
            Gm_list_tmp.append(Gmeasure)

        #求平均保存到k_acc_list中   
        k_average_acc=np.mean(acc_list_tmp)
        k_average_acc=round(k_average_acc,5)
        k_acc_list.append(k_average_acc)

        #求平均保存到k_f1_list中   
        k_average_f1=np.mean(f1_list_tmp)
        k_average_f1=round(k_average_f1,5)
        k_f1_list.append(k_average_f1)

        #求平均保存到k_auc_list中   
        k_average_auc=np.mean(auc_list_tmp)
        k_average_auc=round(k_average_auc,5)
        k_auc_list.append(k_average_auc)
        
        #求平均保存到k_recall_list中   
        k_average_recall=np.mean(recall_list_tmp)
        k_average_recall=round(k_average_recall,5)
        k_recall_list.append(k_average_recall)


        #求平均保存到k_mcc_list中   
        k_average_mcc=np.mean(mcc_list_tmp)
        k_average_mcc=round(k_average_mcc,5)
        k_mcc_list.append(k_average_mcc)

        #求平均保存到k_Gm_list中   
        k_average_Gm=np.mean(Gm_list_tmp)
        k_average_Gm=round(k_average_Gm,5)
        k_Gm_list.append(k_average_Gm)
        print(f'maptiems:{map_times}\tenhancetimes:{enhance_times}\tk_average_acc:{k_average_acc}\tk_average_f1:{k_average_f1}\tk_average_auc:{k_average_auc}\tk_average_recall:{k_average_recall}\tk_average_mcc:{k_average_mcc}\tk_average_Gm:{k_average_Gm}\t')

k_acc_array=np.array(k_acc_list)
k_acc_array=k_acc_array.reshape(5,5)
# 一维最大值索引
#idx_max_ravel = np.argmax(k_acc_array)
# true索引
#idx_max = np.unravel_index(idx_max_ravel, k_acc_array.shape)
#print('max times:',idx_max)

k_f1_array=np.array(k_f1_list)
k_f1_array=k_f1_array.reshape(5,5)

k_auc_array=np.array(k_auc_list)
k_auc_array=k_auc_array.reshape(5,5)

k_recall_array=np.array(k_recall_list)
k_recall_array=k_recall_array.reshape(5,5)

k_mcc_array=np.array(k_mcc_list)
k_mcc_array=k_mcc_array.reshape(5,5)

k_Gm_array=np.array(k_Gm_list)
k_Gm_array=k_Gm_array.reshape(5,5)

sio.savemat('./data_remember/0.75WBLS_Smote_PC4_MEbest.mat',{'acc':k_acc_array,'f1':k_f1_array,'auc':k_auc_array,'recall':k_recall_array,'mcc':k_mcc_array,'Gm':k_Gm_array})

maptiems:15	enhancetimes:15	k_average_acc:0.9	k_average_f1:0.89968	k_average_auc:0.90101	k_average_recall:0.9	k_average_mcc:0.80651	k_average_Gm:0.90101	
maptiems:15	enhancetimes:20	k_average_acc:0.90417	k_average_f1:0.90393	k_average_auc:0.90516	k_average_recall:0.90417	k_average_mcc:0.81382	k_average_Gm:0.90516	
maptiems:15	enhancetimes:25	k_average_acc:0.90729	k_average_f1:0.90707	k_average_auc:0.90842	k_average_recall:0.90729	k_average_mcc:0.82046	k_average_Gm:0.90842	
maptiems:15	enhancetimes:30	k_average_acc:0.90846	k_average_f1:0.90825	k_average_auc:0.90951	k_average_recall:0.90846	k_average_mcc:0.82244	k_average_Gm:0.9095	
maptiems:15	enhancetimes:35	k_average_acc:0.91156	k_average_f1:0.91138	k_average_auc:0.91258	k_average_recall:0.91156	k_average_mcc:0.82828	k_average_Gm:0.91258	
maptiems:20	enhancetimes:15	k_average_acc:0.91042	k_average_f1:0.91023	k_average_auc:0.91123	k_average_recall:0.91042	k_average_mcc:0.82546	k_average_Gm:0.91123	
maptiems:20	enhancetimes:20	k_average